## カテゴリに所属する確率の説明・予測

### 11.1 分析に用いるサンプルデータ

#### 11.1.1 空き巣調査データ
|変数|説明|
|:--|:--|
|空き巣|空き巣被害のあり・なし|
|不在時間|平日9～18時の不在時間(20日間)の平均|
|会話|近隣住民との会話の回数(1週間の合計)|
|築年数|住居の築年数|
|セキュリティ|セキュリティ会社のサービスの加入・非加入|
|飼い犬|飼い犬の有無|

#### 11.1.2 分析目的
空き巣被害にたいして不在時間などがどう影響するか明らかにする.

##### 分析の流れ
- データの整形
- ロジスティック回帰モデルにおける指標算出
- モデル評価
- モデル指標の解釈

#### 11.1.3 ロジスティック回帰の概要とデータの整形
- 回帰分析と異なり,カテゴリカル変数を目的変数として, ある水準となる確率を説明・予測
- カテゴリカル変数は文字データのままでは分析に用いることが出来ず, 数値データに変換する必要がある

In [1]:
#データの読み込み・変換・確認

aks <- read.csv("https://raw.githubusercontent.com/gndb3168/Renshu_data/master/R_MA/Sec_11/burglary_research.csv")

aks$空き巣01 <- ifelse(aks$空き巣=="あり", 1, 0)
aks$セキュリティ01 <- ifelse(aks$セキュリティ=="加入", 1, 0)
aks$飼い犬01 <- ifelse(aks$飼い犬=="あり", 1, 0)

head(aks)

空き巣,不在時間,会話,築年数,セキュリティ,飼い犬,空き巣01,セキュリティ01,飼い犬01
あり,2.92,1,21,加入,なし,1,1,0
なし,1.19,1,18,非加入,なし,0,0,0
あり,5.20,1,6,非加入,なし,1,0,0
あり,6.06,1,17,非加入,なし,1,0,0
あり,5.24,1,24,加入,あり,1,1,1
あり,2.30,1,9,加入,なし,1,1,0


### 11.2 係数・切片の推定と解釈

#### 11.2.1 係数・切片の出力と解釈
ロジスティック回帰には関数glmを用いる.

In [2]:
#ロジスティック回帰の実行
aks.out <- glm(空き巣01~不在時間+会話+築年数+セキュリティ01+飼い犬01, family="binomial", data=aks)

In [3]:
#ロジスティック回帰の結果の出力
summary(aks.out)


Call:
glm(formula = 空き巣01 ~ 不在時間 + 会話 + 築年数 + セキュリティ01 + 
    飼い犬01, family = "binomial", data = aks)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-2.079  -1.002  -0.496   1.019   2.173  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.941239   0.278842  -3.376 0.000737 ***
不在時間        0.393992   0.041904   9.402  < 2e-16 ***
会話           -0.550249   0.130741  -4.209 2.57e-05 ***
築年数          0.005839   0.010774   0.542 0.587818    
セキュリティ01 -0.548210   0.157485  -3.481 0.000499 ***
飼い犬01       -0.480529   0.214167  -2.244 0.024851 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1107.91  on 799  degrees of freedom
Residual deviance:  956.81  on 794  degrees of freedom
AIC: 968.81

Number of Fisher Scoring iterations: 3


係数の推定値が正の場合は, 目的変数の値が1になる確率を上げる影響があることを示す.  
z値は「各係数の母数が0である」という統計的仮設検定における検定統計量の値.  
切片は「全ての説明変数の値が0であるときに, 目的変数の値が1になる確率」

#### 11.2.2 係数・切片の指数変換値の算出と解釈

係数や切片の指数変換を行うと解釈がしやすくなる.

In [4]:
#係数・切片の指数変換値の算出
exp(aks.out$coefficients)

(Intercept)       不在時間           会話         築年数 セキュリティ01 
     0.3901441      1.4828885      0.5768062      1.0058564      0.5779834 
      飼い犬01 
     0.6184561

##### 指数変換した切片と係数の解釈
- 切片: 全説明変数の値が0のとき, 「目的変数が1となる」オッズ
- 係数: 他の説明変数の値を固定したもとで当該説明変数が1増加したとき, 「目的変数が1となる」オッズは何倍になるか

サンプルデータの場合, 切片の指数変換値は「全説明変数の値が0のとき, 空き巣被害が生じるオッズが0.39」. 係数の指数変換値は「他の説明変数の値を固定したもとで, 不在時間が1増加したとき, 空き巣被害が生じるオッズは1.48倍になる」ことを意味する.

##### 確率的な試行の結果がAとなるオッズの求め方
$$
    \frac{\text{Aが生じる確率}}{\text{A以外が生じる確率}}
$$

#### 11.2.3 係数・切片に関する信頼区間の算出

In [5]:
#係数・切片に関する信頼区間の算出
confint(aks.out, level=0.95)

#指数変換した信頼区間の算出
exp(confint(aks.out, level=0.95))

Waiting for profiling to be done...


,2.5 %,97.5 %
(Intercept),-1.49300752,-0.39857705
不在時間,0.31356566,0.47801144
会話,-0.80963933,-0.29647263
築年数,-0.01526572,0.02700569
セキュリティ01,-0.85880279,-0.24101294
飼い犬01,-0.90438019,-0.06341113


Waiting for profiling to be done...


,2.5 %,97.5 %
(Intercept),0.2246959,0.6712746
不在時間,1.3682953,1.6128639
会話,0.4450185,0.7434360
築年数,0.9848502,1.0273736
セキュリティ01,0.4236690,0.7858315
飼い犬01,0.4047927,0.9385575


#### 11.2.4 標準化係数の算出と解釈

目的変数に対する影響の強さを説明変数間で比較したい場合, 単位が不揃いだと解釈が難しくなる. 影響の強さを比較するには全ての量的な説明変数をそれぞれ標準化(z得点化)した結果を利用する(1.4.3 'scale'参照).

In [6]:
#自作関数の実行
#LRAstdcoef(aks.out, c("不在時間", "会話", "築年数"))

### 11.3 モデルの良さの評価

ロジスティック回帰におけるモデル評価は, 主に当てはまりの良さを評価する Hosmer-Lemeshow の適合度検定, 予測の良さの評価としてAIC, BICなどがある.

In [7]:
#Hosmer-Lemeshowの適合度検定

#hoslem.testを利用するためのResourceSelectionライブラリの呼び出し
library(ResourceSelection)
hoslem.test(x=aks.out$y, y=fitted(aks.out))

ResourceSelection 0.3-2 	 2017-02-28



	Hosmer and Lemeshow goodness of fit (GOF) test

data:  aks.out$y, fitted(aks.out)
X-squared = 12.121, df = 8, p-value = 0.1459


- Hosmer-Lemshow の適合度検定における帰無仮説は「モデルが適合している」で, 棄却されなければモデルが当てはまっていると解釈
- 引数xに「目的変数の観測値の変数」, 引数yに「目的変数の予測値の変数」を指定

#### 11.3.2 予測の良さの評価指標の出力と解釈

In [8]:
#AICとBICの算出(3.7参照)
extractAIC(aks.out)  #AIC
extractAIC(aks.out, k=log(nrow(aks.out$data)))   #BIC

[1]   6.0000 968.8111

[1]   6.0000 996.9187

### 11.4 その他の有益な指標

#### 11.4.1 説明変数群の有効性の確認

In [9]:
#説明変数群の有効性の確認
aks.out_null <- glm(空き巣01~1, family="binomial", data=aks)
anova(aks.out_null, aks.out, test="Chisq")

Resid. Df,Resid. Dev,Df,Deviance,Pr(>Chi)
799,1107.9102,NA,NA,NA
794,956.8111,5,151.0992,7.790047e-31


In [10]:
#変数選択の実行
step(aks.out_null, direction="both", scope=(~不在時間+会話+築年数+セキュリティ01+飼い犬01))

Start:  AIC=1109.91
空き巣01 ~ 1

                 Df Deviance     AIC
+ 不在時間        1   992.86  996.86
+ 会話            1  1073.07 1077.07
+ セキュリティ01  1  1099.64 1103.64
<none>               1107.91 1109.91
+ 飼い犬01        1  1105.96 1109.96
+ 築年数          1  1107.69 1111.69

Step:  AIC=996.86
空き巣01 ~ 不在時間

                 Df Deviance     AIC
+ 会話            1   974.08  980.08
+ セキュリティ01  1   980.99  986.99
+ 飼い犬01        1   987.91  993.91
<none>                992.86  996.86
+ 築年数          1   992.49  998.49
- 不在時間        1  1107.91 1109.91

Step:  AIC=980.08
空き巣01 ~ 不在時間 + 会話

                 Df Deviance     AIC
+ セキュリティ01  1   962.22  970.22
+ 飼い犬01        1   969.46  977.46
<none>                974.08  980.08
+ 築年数          1   973.72  981.72
- 会話            1   992.86  996.86
- 不在時間        1  1073.07 1077.07

Step:  AIC=970.22
空き巣01 ~ 不在時間 + 会話 + セキュリティ01

                 Df Deviance     AIC
+ 飼い犬01        1   957.11  967.11
<none>                962.22  970.22
+ 築年数          1  


Call:  glm(formula = 空き巣01 ~ 不在時間 + 会話 + セキュリティ01 + 飼い犬01, 
    family = "binomial", data = aks)

Coefficients:
   (Intercept)        不在時間            会話  セキュリティ01        飼い犬01  
       -0.8658          0.3934         -0.5499         -0.5492         -0.4810  

Degrees of Freedom: 799 Total (i.e. Null);  795 Residual
Null Deviance:	    1108 
Residual Deviance: 957.1 	AIC: 967.1

#### 11.4.3 多重共線性の確認

In [11]:
#多重共線性の確認
library(car)
vif(aks.out)

Loading required package: carData


不在時間           会話         築年数 セキュリティ01       飼い犬01 
      1.032676       1.005094       1.001457       1.018138       1.013356